## Can a machine learning algorithm do a good job of predicting correct quartet configurations?

### imports:

In [1]:
import numpy as np
import toytree
import itertools
import copy
from itertools import compress
import toyplot
import tensorflow as tf
import contextlib
import sys
import io
import ete3
import os
import ipyrad as ip
import subprocess

### functions:

In [290]:
def decompose_to_quartets(tre):

    ## set to compared ordered sets, and set to store quartet
    qrts = set()
    stored = set()

    ## get all tips in tree as a set
    n_all = set(tre.tree.get_leaf_names())

    ## traverse tree
    for node in tre.tree.traverse():
        ## skip root or tip nodes
        if not (node.is_root() or node.is_leaf()):

            ## get all tips below this node
            below = set(node.get_leaf_names())
            above = n_all - below

            ## get all combinations of 2 above and 2 below this node
            for qrt in get_all_combs(tre, above, below):

                ## add to quartet set
                sqrt = tuple(sorted(qrt))
                if sqrt not in qrts:
                    stored.add(qrt)
                    qrts.add(sqrt)

    ## store qrts  
    return stored
def get_all_combs(self, set1, set2, as_list=False):
    #qiter = (sorted(i) + sorted(j) for (i, j) in itertools.product(
    #            itertools.combinations(set1, 2), 
    #            itertools.combinations(set2, 2),
    #        ))
    qiter = (tuple(i+j) for (i, j) in itertools.product(
        itertools.combinations(set1, 2), 
        itertools.combinations(set2, 2),
    ))

    ## option to return as list 
    if as_list:
        return list(qiter)
    ## but returning as generator is more efficient
    else:
        return qiter

## Show that we should be able to recognize the true quartet visually

Here, we import a random tree, a sequence simulated on that tree, and a random quartet from the tree. The tree has random branch lengths, and the sequences paired with the trees evolve at varying rates and are of varying lengths.

In [318]:
# pick a random tree from file
treenum = np.random.choice(range(2000))+1
print "Tree number: " + str(treenum)
thetree = toytree.tree('random_trees/samp'+str(treenum+1)+'.tre')
# get all TRUE splits on the tree
treeqrts = list(decompose_to_quartets(thetree))

fname = ('tree_seqs/test'+str(treenum+1)+'.dat')
with open(fname) as f:
    sequences = f.readlines()
# remove whitespace characters like `\n` at the end of each line
sequences = [x.strip() for x in sequences] 
sequences.pop(0)

# separate all sequences and tip names
names = [sequences[i][0:10].strip(" ") for i in range(len(sequences))]
iso_sequences = [sequences[i][10:].strip(" ") for i in range(len(sequences))]

# pick a random quartet
num = int((np.random.choice(range(len(treeqrts)),1)))
print "Quartet number = " + str(num)

# visualize this quartet on the tree
colors = [thetree.colors[0] if i==True else thetree.colors[1] \
          for i in [thetree.get_node_labels()[q] in treeqrts[num] for q in range(len(thetree.get_node_labels()))]]
thetree.draw(
    width=300,
    height = 490,
    node_labels=False, 
    node_color=colors,
    node_size=15,
);


print "the TRUE split is: " + str(treeqrts[num])

Tree number: 391
Quartet number = 18341
the TRUE split is: ('T11', 'T5', 'T33', 'T17')


T6 T14 T21 T29 T35 T31 T27 T15 T9 T24 T22 T33 T17 T12 T18 T10 T5 T1 T3 T36 T32 T13 T4 T23 T7 T28 T34 T8 T16 T20 T19 T11 T25 T37 T26 T2 T30 idx: 0
name: 0
dist: 4.72947883569
support: 100 idx: 1
name: 1
dist: 0.894314320535
support: 100 idx: 2
name: 2
dist: 1.00825511165
support: 100 idx: 3
name: 3
dist: 1.35997428612
support: 100 idx: 4
name: 4
dist: 0.76543980408
support: 100 idx: 5
name: 5
dist: 0.965287487158
support: 100 idx: 6
name: 6
dist: 0.296317017981
support: 100 idx: 7
name: 7
dist: 1.08035348192
support: 100 idx: 8
name: 8
dist: 0.480195281746
support: 100 idx: 9
name: 9
dist: 0.0186372946789
support: 100 idx: 10
name: 10
dist: 0.244206556386
support: 100 idx: 11
name: 11
dist: 0.658092720381
support: 100 idx: 12
name: 12
dist: 0.0791701534912
support: 100 idx: 13
name: 13
dist: 1.43567016606
support: 100 idx: 14
name: 14
dist: 1.27464492568
support: 100 idx: 15
name: 15
dist: 0.517913193273
support: 100 idx: 16
name: 16
dist: 0.748073668486
support: 100 idx: 17
name: 17
dist: 0.35539469991
support: 100 idx: 18
name: 18
dist: 0.373569688452
support: 100 idx: 19
name: 19
dist: 0.725026674986
support: 100 idx: 20
name: 20
dist: 0.508851067405
support: 100 idx: 21
name: 21
dist: 0.488781352724
support: 100 idx: 22
name: 22
dist: 0.0925437733478
support: 100 idx: 23
name: 23
dist: 0.660425909874
support: 100 idx: 24
name: 24
dist: 0.410468365185
support: 100 idx: 25
name: 25
dist: 0.681760191374
support: 100 idx: 26
name: 26
dist: 1.12855416807
support: 100 idx: 27
name: 27
dist: 0.250681950779
support: 100 idx: 28
name: 28
dist: 0.231234452294
support: 100 idx: 29
name: 29
dist: 0.406740973044
support: 100 idx: 30
name: 30
dist: 0.0522301674928
support: 100 idx: 31
name: 31
dist: 1.55146850317
support: 100 idx: 32
name: 32
dist: 0.0359607856088
support: 100 idx: 33
name: 33
dist: 0.157316374371
support: 100 idx: 34
name: 34
dist: 0.359200560001
support: 100 idx: 35
name: 35
dist: 0.119217634922
support: 100 idx: 36
name: T6
dist: 2.42912722123
support: 100 idx: 37
name: T14
dist: 1.06915293511
support: 100 idx: 38
name: T21
dist: 1.06915293511
support: 100 idx: 39
name: T29
dist: 2.67194252879
support: 100 idx: 40
name: T35
dist: 1.41033802365
support: 100 idx: 41
name: T31
dist: 0.329984541735
support: 100 idx: 42
name: T27
dist: 0.329984541735
support: 100 idx: 43
name: T15
dist: 1.20782246521
support: 100 idx: 44
name: T9
dist: 0.963615908825
support: 100 idx: 45
name: T24
dist: 0.963615908825
support: 100 idx: 46
name: T22
dist: 0.568367039509
support: 100 idx: 47
name: T33
dist: 0.489196886018
support: 100 idx: 48
name: T17
dist: 0.489196886018
support: 100 idx: 49
name: T12
dist: 1.1034683684
support: 100 idx: 50
name: T18
dist: 0.35539469991
support: 100 idx: 51
name: T10
dist: 0.0
support: 100 idx: 52
name: T5
dist: 0.0
support: 100 idx: 53
name: T1
dist: 0.522785198231
support: 100 idx: 54
name: T3
dist: 0.0139341308264
support: 100 idx: 55
name: T36
dist: 0.0139341308264
support: 100 idx: 56
name: T32
dist: 0.759030520493
support: 100 idx: 57
name: T13
dist: 0.00606083727091
support: 100 idx: 58
name: T4
dist: 0.00606083727091
support: 100 idx: 59
name: T23
dist: 0.25601838196
support: 100 idx: 60
name: T7
dist: 0.25601838196
support: 100 idx: 61
name: T28
dist: 1.0857121279
support: 100 idx: 62
name: T34
dist: 0.603795724829
support: 100 idx: 63
name: T8
dist: 0.603795724829
support: 100 idx: 64
name: T16
dist: 0.428289204079
support: 100 idx: 65
name: T20
dist: 0.376059036586
support: 100 idx: 66
name: T19
dist: 0.376059036586
support: 100 idx: 67
name: T11
dist: 0.626837007194
support: 100 idx: 68
name: T25
dist: 0.626837007194
support: 100 idx: 69
name: T37
dist: 0.505481418432
support: 100 idx: 70
name: T26
dist: 0.146280858431
support: 100 idx: 71
name: T2
dist: 0.0270632235094
support: 100 idx: 72
name: T30
dist: 0.0270632235094
support: 100

Let's pretend we don't know the true quartet and are just taking a random set of four tips and shuffling them three ways:

In [319]:
qrtnum = num
true_qrt = np.array(treeqrts[qrtnum])
tipnames = copy.deepcopy(true_qrt)
np.random.shuffle(tipnames)
# is correct config of these tips [0123],[0213], or [0312]
correct_config = [int( ((set([tipnames[i] for i in q[0:2]]) == set(true_qrt[0:2])) or (set([tipnames[i] for i in q[0:2]]) == set(true_qrt[2:4]))) ) for q in [[0,1,2,3],[0,2,1,3],[0,3,1,2]]]
print "The true quartet is: " + str(true_qrt)
print "The random configuration of this (i.e. when true quartet unknown) is: " + str(tipnames)
print "The correct configuration of the random configuration is: " + str([[0,1,2,3],[0,2,1,3], [0,3,1,2]][correct_config.index(1)])


The true quartet is: ['T11' 'T5' 'T33' 'T17']
The random configuration of this (i.e. when true quartet unknown) is: ['T33' 'T11' 'T5' 'T17']
The correct configuration of the random configuration is: [0, 3, 1, 2]


### Now, we can shuffle our random arrangement of the four tips into the three possible splits: [0,1,2,3],[0,2,1,3], and [0,3,1,2]. 

In [320]:
interestednames = tipnames # this is a list of four tip names... e.g. ["t1","t2","t3","t4]
taxa_ids = list(itertools.chain.from_iterable([list(compress(range(len(names)),i)) for i in [[q == i for i in names] for q in interestednames]]))

tempobj = [iso_sequences[i] for i in taxa_ids]

# eliminate non-snps

ind_samples = []
for i in range(len(tempobj[0])):
    currentbase = ([tempobj[q][i] for q in range(len(tempobj))])
    if (len(set(currentbase)) > 1):
        ind_samples.append(currentbase)
ind_samples_reset = ind_samples

# separate sequences by fifth taxon

ind_samples = np.array(ind_samples_reset)
ind_samples = np.where(ind_samples=='A',0,ind_samples)
ind_samples = np.where(ind_samples=='C',1,ind_samples)
ind_samples = np.where(ind_samples=='G',2,ind_samples)
ind_samples = np.where(ind_samples=='T',3,ind_samples)
ind_samples = ind_samples.astype(int)


print(correct_config)
for q in [[0,1,2,3],[0,2,1,3],[0,3,1,2]]:
    # get the matrices
    indexmat = np.array(range(16))
    indexmat.shape=(4,4)
    # order across matrix is 00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33
    fullmat0123 = np.zeros(shape=(16,16))
    arr0123 = ind_samples[:,q]
    for i in range(len(arr0123)):
                # get row number 
        rownum = int(indexmat[arr0123[i][0],arr0123[i][1]])
                # get col number
        colnum = int(indexmat[arr0123[i][2],arr0123[i][3]])
        fullmat0123[rownum,colnum] = fullmat0123[rownum,colnum] + 1
    #images.append(fullmat0123.flatten()/max(fullmat0123.flatten()))
    #labels.append(correct_config)
    toyplot.matrix((fullmat0123.flatten()/max(fullmat0123.flatten())).reshape(16,16))

[0, 0, 1]


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 0.0 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.0 0.545454545455 0.0909090909091 0.0 0.0 0.363636363636 0.0 0.0 0.0 1 0.272727272727 0.0 0.0 0.0 0.636363636364 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.0 2 0.454545454545 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.727272727273 0.0 0.0 0.0 0.0 0.0 0.0 0.0 3 0.818181818182 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.0 0.636363636364 0.0 0.0 0.0 4 0.0909090909091 0.363636363636 0.0 0.0 0.0 0.181818181818 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.0 0.0 0.0 0.0 5 0.0909090909091 0.272727272727 0.0 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.363636363636 0.0 0.0 0.0 0.0909090909091 0.0 0.0 6 0.0 0.181818181818 0.0 0.0 0.0 0.181818181818 0.0 0.0 0.0 0.363636363636 0.0 0.0 0.0 0.0 0.0 0.0 7 0.0 0.0 0.0 0.0 0.0 0.363636363636 0.0 0.0 0.0 0.181818181818 0.0 0.0 0.0 1.0 0.0 0.181818181818 8 0.0 0.0 0.454545454545 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.181818181818 0.0 0.0 0.0 0.0 0.0 9 0.0 0.0 0.272727272727 0.0 0.0 0.0 0.545454545455 0.0 0.0 0.0 0.181818181818 0.0 0.0 0.0 0.0 0.0 10 0.0 0.0 0.363636363636 0.0 0.0 0.0 0.272727272727 0.0 0.0909090909091 0.181818181818 0.0 0.0 0.0 0.0 0.272727272727 0.0 11 0.0 0.0 0.181818181818 0.0 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.181818181818 0.0 0.0 0.0 0.363636363636 0.0 12 0.181818181818 0.0 0.0 0.454545454545 0.0 0.0909090909091 0.0 0.181818181818 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.363636363636 13 0.0 0.0 0.0 0.0 0.0 0.0909090909091 0.0 0.454545454545 0.0 0.0 0.0 0.181818181818 0.0 0.0 0.0 0.181818181818 14 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.181818181818 0.0 0.0 0.0909090909091 0.727272727273 0.0 0.0 0.0 0.454545454545 15 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.0 0.181818181818 0.0 0.0 0.0 0.0909090909091 0.0909090909091 0.0 0.0 0.0

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 0.0 0.0 0.0 0.0 0.272727272727 0.0 0.0 0.0 0.454545454545 0.0 0.0 0.0 0.818181818182 0.0 0.0 0.0 1 0.0909090909091 0.0 0.0 0.0 0.636363636364 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.0 2 0.545454545455 0.0909090909091 0.0 0.0 0.0909090909091 0.0 0.0 0.0 0.727272727273 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.0 3 0.363636363636 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.636363636364 0.0 0.0 0.0 4 0.0909090909091 0.363636363636 0.0 0.0 0.0909090909091 0.272727272727 0.0 0.0 0.0 0.181818181818 0.0 0.0 0.0 0.0 0.0 0.0 5 0.0 0.181818181818 0.0 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.181818181818 0.0 0.0 0.0 0.363636363636 0.0 0.0 6 0.0 0.0909090909091 0.0 0.0 0.0 0.363636363636 0.0 0.0 0.0 0.363636363636 0.0 0.0 0.0 0.181818181818 0.0 0.0 7 0.0 0.0 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.181818181818 8 0.0 0.0 0.454545454545 0.0 0.0 0.0 0.272727272727 0.0 0.0 0.0 0.363636363636 0.0 0.0 0.0 0.181818181818 0.0 9 0.0 0.0 0.0 0.0 0.0 0.0 0.545454545455 0.0 0.0 0.0 0.272727272727 0.0 0.0 0.0 0.0 0.0909090909091 10 0.0 0.0 0.181818181818 0.0 0.0 0.0 0.181818181818 0.0 0.0909090909091 0.181818181818 0.0 0.0 0.0 0.0 0.181818181818 0.0 11 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.272727272727 0.0 0.0 0.0 0.363636363636 0.0 12 0.181818181818 0.0 0.0 0.454545454545 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0909090909091 13 0.0 0.0909090909091 0.0 0.181818181818 0.0 0.0909090909091 0.0 0.454545454545 0.0 0.0 0.0 0.181818181818 0.0 0.0 0.0 0.181818181818 14 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.181818181818 0.0 0.0 0.0909090909091 0.727272727273 0.0 0.0 0.0 0.0909090909091 15 0.0 0.0 0.0 0.363636363636 0.0 0.0 0.0 0.181818181818 0.0 0.0 0.0 0.454545454545 0.0909090909091 0.0 0.0 0.0

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 0.0 0.0909090909091 0.545454545455 0.363636363636 0.272727272727 0.636363636364 0.0909090909091 0.0909090909091 0.454545454545 0.0 0.727272727273 0.0 0.818181818182 0.0909090909091 0.0909090909091 0.636363636364 1 0.0 0.0 0.0909090909091 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 2 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 3 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 4 0.0909090909091 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 5 0.363636363636 0.181818181818 0.0909090909091 0.0 0.272727272727 0.0 0.363636363636 0.0909090909091 0.181818181818 0.181818181818 0.363636363636 0.0 0.0 0.363636363636 0.181818181818 1.0 6 0.0 0.0 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 7 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.181818181818 8 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.0 0.0 0.0 9 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.181818181818 0.0 0.0 0.0 0.0 0.0 10 0.454545454545 0.0 0.181818181818 0.0 0.272727272727 0.545454545455 0.181818181818 0.0 0.363636363636 0.272727272727 0.0 0.272727272727 0.181818181818 0.0 0.181818181818 0.363636363636 11 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0909090909091 0.0 0.0 12 0.181818181818 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0909090909091 13 0.0 0.0909090909091 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 14 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0909090909091 0.0 0.0 0.0 0.0 0.0 15 0.454545454545 0.181818181818 0.0 0.363636363636 0.0 0.454545454545 0.181818181818 0.181818181818 0.0 0.181818181818 0.727272727273 0.454545454545 0.0909090909091 0.181818181818 0.0909090909091 0.0

We see that there are structured differences here -- maybe we can use a model to classify them.

# Training a machine learning model

We can just adapt the above code to train a machine learning model by showing it many different incorrect vs. correct matrices.

In [283]:
# functions to generate a random correctly / incorrectly configured quartet matrices
def random_correct_matrix():
    # pick a random tree from file
    treenum = np.random.choice(range(2000))+1
    #print "Tree number: " + str(treenum)
    thetree = toytree.tree('random_trees/samp'+str(treenum+1)+'.tre')
    # get all TRUE splits on the tree
    treeqrts = list(decompose_to_quartets(thetree))

    fname = ('tree_seqs/test'+str(treenum+1)+'.dat')
    with open(fname) as f:
        sequences = f.readlines()
    # remove whitespace characters like `\n` at the end of each line
    sequences = [x.strip() for x in sequences] 
    sequences.pop(0)

    # separate all sequences and tip names
    names = [sequences[i][0:10].strip(" ") for i in range(len(sequences))]
    iso_sequences = [sequences[i][10:].strip(" ") for i in range(len(sequences))]

    # pick a random quartet
    num = int((np.random.choice(range(len(treeqrts)),1)))
    #print "Quartet number = " + str(num)

    qrtnum = num
    true_qrt = np.array(treeqrts[qrtnum])
    correct_config = [1,0,0]
    #print "The true quartet is: " + str(true_qrt)
    #print "The random configuration of this (i.e. when true quartet unknown) is: " + str(tipnames)
    #print "The correct configuration of the random configuration is: " + str([[0,1,2,3],[0,2,1,3], [0,3,1,2]][correct_config.index(1)])
    
    interestednames = copy.deepcopy(true_qrt) # this is a list of four tip names... e.g. ["t1","t2","t3","t4]
    taxa_ids = list(itertools.chain.from_iterable([list(compress(range(len(names)),i)) for i in [[q == i for i in names] for q in interestednames]]))

    tempobj = [iso_sequences[i] for i in taxa_ids]

    # eliminate non-snps

    ind_samples = []
    for i in range(len(tempobj[0])):
        currentbase = ([tempobj[q][i] for q in range(len(tempobj))])
        if (len(set(currentbase)) > 1):
            ind_samples.append(currentbase)
    ind_samples_reset = ind_samples

    # separate sequences by fifth taxon

    ind_samples = np.array(ind_samples_reset)
    ind_samples = np.where(ind_samples=='A',0,ind_samples)
    ind_samples = np.where(ind_samples=='C',1,ind_samples)
    ind_samples = np.where(ind_samples=='G',2,ind_samples)
    ind_samples = np.where(ind_samples=='T',3,ind_samples)
    ind_samples = ind_samples.astype(int)

    #print(correct_config)
    # get the matrices
    indexmat = np.array(range(16))
    indexmat.shape=(4,4)
    # order across matrix is 00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33
    fullmat0123 = np.zeros(shape=(16,16))
    arr0123 = ind_samples # using the correct config
    for i in range(len(arr0123)):
                # get row number 
        rownum = int(indexmat[arr0123[i][0],arr0123[i][1]])
                # get col number
        colnum = int(indexmat[arr0123[i][2],arr0123[i][3]])
        fullmat0123[rownum,colnum] = fullmat0123[rownum,colnum] + 1
    #images.append(fullmat0123.flatten()/max(fullmat0123.flatten()))
    #labels.append(correct_config)
    return (fullmat0123.flatten()/max(fullmat0123.flatten()))


def random_wrong_matrix():
    #pick a random tree from file
    treenum = np.random.choice(range(2000))+1
    #print "Tree number: " + str(treenum)
    thetree = toytree.tree('random_trees/samp'+str(treenum+1)+'.tre')
    # get all TRUE splits on the tree
    treeqrts = list(decompose_to_quartets(thetree))

    fname = ('tree_seqs/test'+str(treenum+1)+'.dat')
    with open(fname) as f:
        sequences = f.readlines()
    # remove whitespace characters like `\n` at the end of each line
    sequences = [x.strip() for x in sequences] 
    sequences.pop(0)

    # separate all sequences and tip names
    names = [sequences[i][0:10].strip(" ") for i in range(len(sequences))]
    iso_sequences = [sequences[i][10:].strip(" ") for i in range(len(sequences))]

    # pick a random quartet
    num = int((np.random.choice(range(len(treeqrts)),1)))
    #print "Quartet number = " + str(num)

    qrtnum = num
    true_qrt = np.array(treeqrts[qrtnum])
    correct_config = [1,0,0]
    #print "The true quartet is: " + str(true_qrt)
    #print "The random configuration of this (i.e. when true quartet unknown) is: " + str(tipnames)
    #print "The correct configuration of the random configuration is: " + str([[0,1,2,3],[0,2,1,3], [0,3,1,2]][correct_config.index(1)])
    
    interestednames = copy.deepcopy(true_qrt) # this is a list of four tip names... e.g. ["t1","t2","t3","t4]
    taxa_ids = list(itertools.chain.from_iterable([list(compress(range(len(names)),i)) for i in [[q == i for i in names] for q in interestednames]]))

    tempobj = [iso_sequences[i] for i in taxa_ids]

    # eliminate non-snps

    ind_samples = []
    for i in range(len(tempobj[0])):
        currentbase = ([tempobj[q][i] for q in range(len(tempobj))])
        if (len(set(currentbase)) > 1):
            ind_samples.append(currentbase)
    ind_samples_reset = ind_samples

    # separate sequences by fifth taxon

    ind_samples = np.array(ind_samples_reset)
    ind_samples = np.where(ind_samples=='A',0,ind_samples)
    ind_samples = np.where(ind_samples=='C',1,ind_samples)
    ind_samples = np.where(ind_samples=='G',2,ind_samples)
    ind_samples = np.where(ind_samples=='T',3,ind_samples)
    ind_samples = ind_samples.astype(int)


    #print(correct_config)
    # get the matrices
    indexmat = np.array(range(16))
    indexmat.shape=(4,4)
    # order across matrix is 00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33
    fullmat0123 = np.zeros(shape=(16,16))
    arr0123 = ind_samples[:,[[0,2,1,3],[0,3,1,2]][np.random.binomial(1,.5)]] # pick one of the wrong configs
    for i in range(len(arr0123)):
                # get row number 
        rownum = int(indexmat[arr0123[i][0],arr0123[i][1]])
                # get col number
        colnum = int(indexmat[arr0123[i][2],arr0123[i][3]])
        fullmat0123[rownum,colnum] = fullmat0123[rownum,colnum] + 1
    #images.append(fullmat0123.flatten()/max(fullmat0123.flatten()))
    #labels.append(correct_config)
    return (fullmat0123.flatten()/max(fullmat0123.flatten()))


## Now we can build a training set of vectorized matrices and accompanying "correct" or "incorrect" labels

In [284]:
images = []
labels = []
for loopnum in range(1000):
    if np.random.binomial(1,.5):
        images.append(random_correct_matrix())
        labels.append([1,0])
    else:
        images.append(random_wrong_matrix())
        labels.append([0,1])

## Now we train and test our model

In [287]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, 256])
W = tf.Variable(tf.zeros([256, 2]))
b = tf.Variable(tf.zeros([2]))

y = tf.nn.softmax(tf.matmul(x, W) + b)

y_ = tf.placeholder(tf.float32, [None, 2])

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

train_step = tf.train.GradientDescentOptimizer(0.6).minimize(cross_entropy)

saver = tf.train.Saver()

init = tf.global_variables_initializer()
# Launch the graph
with tf.Session() as sess:
    sess.run(init)


    #sess = tf.InteractiveSession()
    #tf.global_variables_initializer().run()

    for _ in range(1000):
      batch = np.random.choice(800, 50)
      batch_xs, batch_ys = np.array([images[i] for i in batch]),np.array([labels[i] for i in batch])
      sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={x: images[800:1000], y_: labels[800:1000]}))
    sess.run(accuracy, feed_dict={x: images[800:1000], y_: labels[800:1000]})

0.98


## Do we see the same structures in real data? Example with mammals.

In [289]:
# functions
def get_patterns(goodbases):
    patterns = np.empty(shape = (0,4))
    for base in range(len(goodbases)):
        d = {ni: indi for indi, ni in enumerate(set(goodbases[base]))}
        patterns = np.vstack([patterns,(np.vectorize(d.__getitem__)(goodbases[base]))])
    return(patterns.astype(int))

def most_freq_pattern(the_patterns):
    unique_patterns, freqs  = np.unique(the_patterns,axis = 0,return_counts=True)
    return unique_patterns[np.argmax(freqs)]

def f(genes_alltaxa,geneidx,fourtaxa):
        currentgene = [[genes_alltaxa[geneidx][base][taxon] for taxon in fourtaxa] for base in range(len(genes_alltaxa[geneidx]))]
        return np.array(currentgene)

def exclude(fullgene):
    return np.array([sum(fullgene[q])<= 12 and len(set(fullgene[q])) > 1 for q in xrange(len(fullgene))])

# Making matrices

totalseqs = np.genfromtxt("download_simseqs/concat_mammal_genes.gz",dtype='str')
snpmap = np.loadtxt("download_simseqs/concat_mammal_map.gz").astype(int)

totalseqs = totalseqs.view(np.uint8)
totalseqs = np.where(totalseqs==65,0,totalseqs)
totalseqs = np.where(totalseqs==67,1,totalseqs)
totalseqs = np.where(totalseqs==71,2,totalseqs)
totalseqs = np.where(totalseqs==84,3,totalseqs)
genes_alltaxa = [totalseqs[snpmap[0][i]:snpmap[1][i]] for i in range(len(snpmap[0]))]
alltipcombns=np.array(list(itertools.combinations(range(len(totalseqs[0])), 4)))
alltipcombns = alltipcombns.astype(int)

combocounter = 0
orig_file = np.empty(shape = (0,4))
#    np.savetxt(output_path,orig_file)

targetlen = len(alltipcombns)



allpredictedquarts = np.empty(shape = (0,4))
savecounter = 0 # this will be reset

# set your current combination of four taxa
fourtaxa= alltipcombns[np.random.choice(range(targetlen))]


reducedgene = np.empty(shape = (0,4))
for geneidx in range(len(genes_alltaxa)):
    fullgene = f(genes_alltaxa,geneidx,fourtaxa)
    goodbases = fullgene[exclude(fullgene)]
    if len(goodbases) > 0:
        the_patterns = get_patterns(goodbases)
        indices = np.where((the_patterns == most_freq_pattern(the_patterns)).all(axis=1))[0]
        reducedgene = np.vstack([reducedgene,goodbases[int(np.random.choice(indices,1))]])
        reducedgene = np.vstack([reducedgene,goodbases[np.random.choice(range(len(goodbases)),10)]])
#    print(geneidx)
# make index matrix for each pair of bases. This assigns row / col number for full 16x16 matrix
indexmat = np.array(range(16))
indexmat.shape=(4,4)

        # make 16x16 matrix of zeroes
        # order across matrix is 00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33
        # not good use of space
three_possible = []
possible_configs = [[0,1,2,3],[0,2,1,3],[0,3,1,2]]
arr0123 = copy.deepcopy(reducedgene)
arr0123 = arr0123.astype(int)
for q in possible_configs:
    temp_rearrangement = arr0123[:,q]
    fullmat0123 = np.zeros(shape=(16,16))
    for i in range(len(temp_rearrangement)):
                # get row number 
        rownum = int(indexmat[temp_rearrangement[i][0:2][0],temp_rearrangement[i][0:2][1]])
                # get col number
        colnum = int(indexmat[temp_rearrangement[i][2:4][0],temp_rearrangement[i][2:4][1]])
        fullmat0123[rownum,colnum] = fullmat0123[rownum,colnum] + 1
    three_possible.append((fullmat0123.flatten()/max(fullmat0123.flatten())))
print "Four taxa: " + str(fourtaxa)
toyplot.matrix(three_possible[0].reshape(16,16))
toyplot.matrix(three_possible[1].reshape(16,16))
toyplot.matrix(three_possible[2].reshape(16,16))


Four taxa: [ 5 29 34 36]


(<toyplot.canvas.Canvas at 0x118ffca50>,
 <toyplot.coordinates.Table at 0x118ffc150>)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 0.0 0.412017167382 0.74678111588 0.206008583691 0.416309012876 0.158798283262 0.0515021459227 0.0386266094421 0.888412017167 0.0729613733906 0.437768240343 0.0343347639485 0.326180257511 0.0729613733906 0.0987124463519 0.0987124463519 1 0.115879828326 0.0300429184549 0.00858369098712 0.0 0.00858369098712 0.0214592274678 0.0171673819742 0.0 0.0257510729614 0.0 0.0128755364807 0.0 0.00429184549356 0.0214592274678 0.00858369098712 0.0128755364807 2 0.248927038627 0.0386266094421 0.163090128755 0.0214592274678 0.0300429184549 0.0343347639485 0.0214592274678 0.0128755364807 0.111587982833 0.0214592274678 0.317596566524 0.0128755364807 0.0429184549356 0.0128755364807 0.0214592274678 0.0429184549356 3 0.0257510729614 0.0128755364807 0.0128755364807 0.00429184549356 0.00858369098712 0.0300429184549 0.00858369098712 0.0128755364807 0.00429184549356 0.00429184549356 0.00858369098712 0.00429184549356 0.0 0.00429184549356 0.0128755364807 0.0300429184549 4 0.0600858369099 0.0 0.0128755364807 0.0 0.0171673819742 0.0429184549356 0.00858369098712 0.0 0.00429184549356 0.00429184549356 0.0 0.0 0.0257510729614 0.0171673819742 0.00429184549356 0.0257510729614 5 0.167381974249 0.480686695279 0.0815450643777 0.0386266094421 0.115879828326 0.0 0.18025751073 0.49356223176 0.0557939914163 0.296137339056 0.0858369098712 0.0429184549356 0.0472103004292 1.0 0.0386266094421 0.360515021459 6 0.0171673819742 0.00858369098712 0.0128755364807 0.00858369098712 0.00429184549356 0.0429184549356 0.00429184549356 0.00858369098712 0.00429184549356 0.0214592274678 0.0472103004292 0.0 0.0 0.0128755364807 0.0 0.00429184549356 7 0.0171673819742 0.0343347639485 0.00858369098712 0.0300429184549 0.0 0.257510729614 0.0171673819742 0.0729613733906 0.00429184549356 0.0386266094421 0.00429184549356 0.0128755364807 0.0300429184549 0.124463519313 0.00429184549356 0.214592274678 8 0.231759656652 0.0214592274678 0.0858369098712 0.0171673819742 0.0171673819742 0.0128755364807 0.0128755364807 0.00429184549356 0.111587982833 0.00429184549356 0.154506437768 0.00429184549356 0.0171673819742 0.0171673819742 0.0343347639485 0.0257510729614 9 0.0128755364807 0.0128755364807 0.0128755364807 0.00429184549356 0.00429184549356 0.0214592274678 0.0128755364807 0.00429184549356 0.0171673819742 0.0171673819742 0.0557939914163 0.00858369098712 0.0 0.00429184549356 0.00429184549356 0.00429184549356 10 0.377682403433 0.0472103004292 0.922746781116 0.0257510729614 0.0429184549356 0.0815450643777 0.206008583691 0.0386266094421 0.424892703863 0.188841201717 0.0 0.107296137339 0.0343347639485 0.0386266094421 0.227467811159 0.0772532188841 11 0.00858369098712 0.0 0.00429184549356 0.00429184549356 0.0 0.0128755364807 0.00858369098712 0.0214592274678 0.0 0.00429184549356 0.0472103004292 0.0128755364807 0.00429184549356 0.00858369098712 0.0257510729614 0.0343347639485 12 0.0257510729614 0.0171673819742 0.00858369098712 0.0128755364807 0.00429184549356 0.0128755364807 0.0 0.00858369098712 0.00429184549356 0.0 0.00858369098712 0.00858369098712 0.00429184549356 0.0128755364807 0.00429184549356 0.0515021459227 13 0.0300429184549 0.0343347639485 0.0257510729614 0.0171673819742 0.00858369098712 0.291845493562 0.0128755364807 0.111587982833 0.0 0.0171673819742 0.00858369098712 0.0214592274678 0.0128755364807 0.218884120172 0.0300429184549 0.377682403433 14 0.0171673819742 0.0171673819742 0.0214592274678 0.0171673819742 0.00858369098712 0.0214592274678 0.0 0.0 0.00429184549356 0.00429184549356 0.0214592274678 0.0128755364807 0.00858369098712 0.0171673819742 0.00429184549356 0.0386266094421 15 0.137339055794 0.12017167382 0.0600858369099 0.278969957082 0.0515021459227 0.635193133047 0.0429184549356 0.802575107296 0.0600858369099 0.068669527897 0.0815450643777 0.175965665236 0.111587982833 0.854077253219 0.163090128755 0.0

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 0.0 0.412017167382 0.74678111588 0.206008583691 0.115879828326 0.0300429184549 0.00858369098712 0.0 0.248927038627 0.0386266094421 0.163090128755 0.0214592274678 0.0257510729614 0.0128755364807 0.0128755364807 0.00429184549356 1 0.416309012876 0.158798283262 0.0515021459227 0.0386266094421 0.00858369098712 0.0214592274678 0.0171673819742 0.0 0.0300429184549 0.0343347639485 0.0214592274678 0.0128755364807 0.00858369098712 0.0300429184549 0.00858369098712 0.0128755364807 2 0.888412017167 0.0729613733906 0.437768240343 0.0343347639485 0.0257510729614 0.0 0.0128755364807 0.0 0.111587982833 0.0214592274678 0.317596566524 0.0128755364807 0.00429184549356 0.00429184549356 0.00858369098712 0.00429184549356 3 0.326180257511 0.0729613733906 0.0987124463519 0.0987124463519 0.00429184549356 0.0214592274678 0.00858369098712 0.0128755364807 0.0429184549356 0.0128755364807 0.0214592274678 0.0429184549356 0.0 0.00429184549356 0.0128755364807 0.0300429184549 4 0.0600858369099 0.0 0.0128755364807 0.0 0.167381974249 0.480686695279 0.0815450643777 0.0386266094421 0.0171673819742 0.00858369098712 0.0128755364807 0.00858369098712 0.0171673819742 0.0343347639485 0.00858369098712 0.0300429184549 5 0.0171673819742 0.0429184549356 0.00858369098712 0.0 0.115879828326 0.0 0.18025751073 0.49356223176 0.00429184549356 0.0429184549356 0.00429184549356 0.00858369098712 0.0 0.257510729614 0.0171673819742 0.0729613733906 6 0.00429184549356 0.00429184549356 0.0 0.0 0.0557939914163 0.296137339056 0.0858369098712 0.0429184549356 0.00429184549356 0.0214592274678 0.0472103004292 0.0 0.00429184549356 0.0386266094421 0.00429184549356 0.0128755364807 7 0.0257510729614 0.0171673819742 0.00429184549356 0.0257510729614 0.0472103004292 1.0 0.0386266094421 0.360515021459 0.0 0.0128755364807 0.0 0.00429184549356 0.0300429184549 0.124463519313 0.00429184549356 0.214592274678 8 0.231759656652 0.0214592274678 0.0858369098712 0.0171673819742 0.0128755364807 0.0128755364807 0.0128755364807 0.00429184549356 0.377682403433 0.0472103004292 0.922746781116 0.0257510729614 0.00858369098712 0.0 0.00429184549356 0.00429184549356 9 0.0171673819742 0.0128755364807 0.0128755364807 0.00429184549356 0.00429184549356 0.0214592274678 0.0128755364807 0.00429184549356 0.0429184549356 0.0815450643777 0.206008583691 0.0386266094421 0.0 0.0128755364807 0.00858369098712 0.0214592274678 10 0.111587982833 0.00429184549356 0.154506437768 0.00429184549356 0.0171673819742 0.0171673819742 0.0557939914163 0.00858369098712 0.424892703863 0.188841201717 0.0 0.107296137339 0.0 0.00429184549356 0.0472103004292 0.0128755364807 11 0.0171673819742 0.0171673819742 0.0343347639485 0.0257510729614 0.0 0.00429184549356 0.00429184549356 0.00429184549356 0.0343347639485 0.0386266094421 0.227467811159 0.0772532188841 0.00429184549356 0.00858369098712 0.0257510729614 0.0343347639485 12 0.0257510729614 0.0171673819742 0.00858369098712 0.0128755364807 0.0300429184549 0.0343347639485 0.0257510729614 0.0171673819742 0.0171673819742 0.0171673819742 0.0214592274678 0.0171673819742 0.137339055794 0.12017167382 0.0600858369099 0.278969957082 13 0.00429184549356 0.0128755364807 0.0 0.00858369098712 0.00858369098712 0.291845493562 0.0128755364807 0.111587982833 0.00858369098712 0.0214592274678 0.0 0.0 0.0515021459227 0.635193133047 0.0429184549356 0.802575107296 14 0.00429184549356 0.0 0.00858369098712 0.00858369098712 0.0 0.0171673819742 0.00858369098712 0.0214592274678 0.00429184549356 0.00429184549356 0.0214592274678 0.0128755364807 0.0600858369099 0.068669527897 0.0815450643777 0.175965665236 15 0.00429184549356 0.0128755364807 0.00429184549356 0.0515021459227 0.0128755364807 0.218884120172 0.0300429184549 0.377682403433 0.00858369098712 0.0171673819742 0.00429184549356 0.0386266094421 0.111587982833 0.854077253219 0.163090128755 0.0

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 0.0 0.416309012876 0.888412017167 0.326180257511 0.115879828326 0.00858369098712 0.0257510729614 0.00429184549356 0.248927038627 0.0300429184549 0.111587982833 0.0429184549356 0.0257510729614 0.00858369098712 0.00429184549356 0.0 1 0.412017167382 0.158798283262 0.0729613733906 0.0729613733906 0.0300429184549 0.0214592274678 0.0 0.0214592274678 0.0386266094421 0.0343347639485 0.0214592274678 0.0128755364807 0.0128755364807 0.0300429184549 0.00429184549356 0.00429184549356 2 0.74678111588 0.0515021459227 0.437768240343 0.0987124463519 0.00858369098712 0.0171673819742 0.0128755364807 0.00858369098712 0.163090128755 0.0214592274678 0.317596566524 0.0214592274678 0.0128755364807 0.00858369098712 0.00858369098712 0.0128755364807 3 0.206008583691 0.0386266094421 0.0343347639485 0.0987124463519 0.0 0.0 0.0 0.0128755364807 0.0214592274678 0.0128755364807 0.0128755364807 0.0429184549356 0.00429184549356 0.0128755364807 0.00429184549356 0.0300429184549 4 0.0600858369099 0.0171673819742 0.00429184549356 0.0257510729614 0.167381974249 0.115879828326 0.0557939914163 0.0472103004292 0.0171673819742 0.00429184549356 0.00429184549356 0.0 0.0171673819742 0.0 0.00429184549356 0.0300429184549 5 0.0 0.0429184549356 0.00429184549356 0.0171673819742 0.480686695279 0.0 0.296137339056 1.0 0.00858369098712 0.0429184549356 0.0214592274678 0.0128755364807 0.0343347639485 0.257510729614 0.0386266094421 0.124463519313 6 0.0128755364807 0.00858369098712 0.0 0.00429184549356 0.0815450643777 0.18025751073 0.0858369098712 0.0386266094421 0.0128755364807 0.00429184549356 0.0472103004292 0.0 0.00858369098712 0.0171673819742 0.00429184549356 0.00429184549356 7 0.0 0.0 0.0 0.0257510729614 0.0386266094421 0.49356223176 0.0429184549356 0.360515021459 0.00858369098712 0.00858369098712 0.0 0.00429184549356 0.0300429184549 0.0729613733906 0.0128755364807 0.214592274678 8 0.231759656652 0.0171673819742 0.111587982833 0.0171673819742 0.0128755364807 0.00429184549356 0.0171673819742 0.0 0.377682403433 0.0429184549356 0.424892703863 0.0343347639485 0.00858369098712 0.0 0.0 0.00429184549356 9 0.0214592274678 0.0128755364807 0.00429184549356 0.0171673819742 0.0128755364807 0.0214592274678 0.0171673819742 0.00429184549356 0.0472103004292 0.0815450643777 0.188841201717 0.0386266094421 0.0 0.0128755364807 0.00429184549356 0.00858369098712 10 0.0858369098712 0.0128755364807 0.154506437768 0.0343347639485 0.0128755364807 0.0128755364807 0.0557939914163 0.00429184549356 0.922746781116 0.206008583691 0.0 0.227467811159 0.00429184549356 0.00858369098712 0.0472103004292 0.0257510729614 11 0.0171673819742 0.00429184549356 0.00429184549356 0.0257510729614 0.00429184549356 0.00429184549356 0.00858369098712 0.00429184549356 0.0257510729614 0.0386266094421 0.107296137339 0.0772532188841 0.00429184549356 0.0214592274678 0.0128755364807 0.0343347639485 12 0.0257510729614 0.00429184549356 0.00429184549356 0.00429184549356 0.0300429184549 0.00858369098712 0.0 0.0128755364807 0.0171673819742 0.00858369098712 0.00429184549356 0.00858369098712 0.137339055794 0.0515021459227 0.0600858369099 0.111587982833 13 0.0171673819742 0.0128755364807 0.0 0.0128755364807 0.0343347639485 0.291845493562 0.0171673819742 0.218884120172 0.0171673819742 0.0214592274678 0.00429184549356 0.0171673819742 0.12017167382 0.635193133047 0.068669527897 0.854077253219 14 0.00858369098712 0.0 0.00858369098712 0.00429184549356 0.0257510729614 0.0128755364807 0.00858369098712 0.0300429184549 0.0214592274678 0.0 0.0214592274678 0.00429184549356 0.0600858369099 0.0429184549356 0.0815450643777 0.163090128755 15 0.0128755364807 0.00858369098712 0.00858369098712 0.0515021459227 0.0171673819742 0.111587982833 0.0214592274678 0.377682403433 0.0171673819742 0.0 0.0128755364807 0.0386266094421 0.278969957082 0.802575107296 0.175965665236 0.0